In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score,precision_score,recall_score,roc_curve,classification_report

import matplotlib.pyplot as plt
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')
df.head().T
categorical_columns=list(df.columns[df.dtypes=='object'])
numerical_columns=list(df.columns[df.dtypes!='object'])
df.isna().sum()
df[categorical_columns] = df[categorical_columns].fillna('NA')
df[numerical_columns] = df[numerical_columns].fillna(0)
df.isna().sum()
df_full_train,df_test=train_test_split(df,test_size=0.2,random_state=1)
y_test = df_test.converted.values
del df_test['converted']

if 'converted' in numerical_columns:
    numerical_columns.remove('converted')

def train(df_train,y_train,C=1.0):
    dicts = df_train[categorical_columns+numerical_columns].to_dict(orient ='records')
    dv = DictVectorizer(sparse=False)
    X_train=dv.fit_transform(dicts)
    model = LogisticRegression(C=C,max_iter=5000)
    model.fit(X_train,y_train)
    return dv,model

def predict(df,dv,model):
    dicts =df[categorical_columns+numerical_columns].to_dict(orient='records')
    X=dv.transform(dicts)
    y_pred = model.predict_proba(X)[:,1]
    y_pred_bin = model.predict(X)
    return y_pred,y_pred_bin

C = 1.0
n_splits = 5

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)
#t=[]
scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.converted.values
    y_val = df_val.converted.values

    dv, model = train(df_train, y_train, C=C)
    y_pred,y_pred_bin = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))


C=1.0 0.921 +- 0.011


In [11]:
dv, model = train(df_full_train, df_full_train.converted.values, C=1.0)
y_pred,y_pred_bin = predict(df_test, dv, model)

auc = roc_auc_score(y_test, y_pred)

accuracy_score(y_test,y_pred_bin)
    

0.863481228668942